Universal Functions

In [67]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
from IPython.display import display, HTML
from tqdm import tqdm
import time
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

def print_table(matrix):
    for row in matrix:
        for element in row:
            print(element, end='\t')  # Separate elements by a tab (or any delimiter you prefer)
        print()  # Move to the next line for the next row

# retry mechanism
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
def make_request(url):
    session = requests.Session()
    retry_strategy = Retry(
        total=3,
        status_forcelist=[500, 502, 503, 504],
        allowed_methods=["GET", "POST"],
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    try:
        response = session.get(url, headers = headers)
        response.raise_for_status()
        return response.text
    except:
        return ''

def modern_style_scrape(link):
    response = make_request(link)
    soup = BeautifulSoup(response, 'html.parser')

    if response == '':
        # print(link)
        driver.get(link)
        # driver.implicitly_wait(2) # wait a bit
        page_source = driver.page_source
        soup = BeautifulSoup(page_source)
        driver.quit()

    individual_stats = soup.find('section', id='individual-stats')
    tables = individual_stats.find_all('table')

    score_table = soup.find('table')
    score_table = score_table.find_all('td')
    for i in range(1, len(score_table)):
        try:
            x = int(score_table[i+1].text)
        except:
            ascore = float(score_table[i].text)
            home_team = score_table[i+1].find_all('span', class_='hide-on-small-down')[0].get_text().strip().lower()
            break

    away_team = score_table[0].find_all('span', class_='hide-on-small-down')[0].get_text().strip().lower()
    hscore = float(score_table[-1].text)

    home_team = home_team.replace("Winner", "")
    away_team = away_team.replace("Winner", "")

    if (home_team == 'texas' and hscore > ascore) or (away_team == 'texas' and hscore < ascore):
        tex_win = "Win"
    elif (home_team == 'texas' and hscore < ascore) or (away_team == 'texas' and hscore > ascore):
        tex_win = "Loss"
    else:
        tex_win = "Tie"

    # get date
    big_html = soup.text
    date_index = big_html.find('Date:')
    date_endex = big_html.find('Site:')
    date = big_html[date_index + 6: date_endex].strip()
    date = datetime.strptime(date, "%m/%d/%Y")

    # make gameid
    gameid = away_team.replace(" ", "").lower() + '_' + home_team.replace(" ", "").lower() + '_' + str(date.month) + '_' + str(date.day) + '_' + str(date.year)

    if home_team == 'texas':
        tex_pass = tables[1]
        tex_rush = tables[3]
        tex_rec = tables[5]
    else:
        tex_pass = tables[0]
        tex_rush = tables[2]
        tex_rec = tables[4]

    tex_pass_stats = tex_pass.find_all('td')
    for i in range(len(tex_pass_stats)):  # convert passers to text
        tex_pass_stats[i] = tex_pass_stats[i].text.strip()
    passer_temp = []
    tex_pass_stats_final = []
    for i in range(len(tex_pass_stats)):
        passer_temp.append(tex_pass_stats[i])
        if len(passer_temp)/8 == 1:
            tex_pass_stats_final.append(passer_temp)
            passer_temp = []
    for i in range(len(tex_pass_stats_final)):
        for j in range(1, len(tex_pass_stats_final[i])):
            tex_pass_stats_final[i][j] = float(tex_pass_stats_final[i][j])
    tex_pass_stats_final = pd.DataFrame(tex_pass_stats_final)
    tex_pass_stats_final.columns = ['Player', 'Completions', 'Pass Attempts', 'Pass Yards', 'Passing TDs', 'Interceptions', 'Longest Pass', 'Sacks Taken']

    tex_rush_stats = tex_rush.find_all('td')
    for i in range(len(tex_rush_stats)):  # convert passers to text
        tex_rush_stats[i] = tex_rush_stats[i].text.strip()
    rusher_temp = []
    tex_rush_stats_final = []
    for i in range(len(tex_rush_stats)):
        rusher_temp.append(tex_rush_stats[i])
        if len(rusher_temp)/8 == 1:
            tex_rush_stats_final.append(rusher_temp)
            rusher_temp = []
    for i in range(len(tex_rush_stats_final)):
        for j in range(1, len(tex_rush_stats_final[i])):
            tex_rush_stats_final[i][j] = float(tex_rush_stats_final[i][j])
    tex_rush_stats_final = pd.DataFrame(tex_rush_stats_final)
    tex_rush_stats_final.columns = ['Player', 'Rush Attempts', 'Rush Yards Gained', 'Rush Yards Lost', 'Net Rush Yards', 'Rushing TDs', 'Longest Rush', 'Yards Per Rush']

    tex_rec_stats = tex_rec.find_all('td')
    for i in range(len(tex_rec_stats)):  # convert passers to text
        tex_rec_stats[i] = tex_rec_stats[i].text.strip()
    recer_temp = []
    tex_rec_stats_final = []
    for i in range(len(tex_rec_stats)):
        recer_temp.append(tex_rec_stats[i])
        if len(recer_temp)/5 == 1:
            tex_rec_stats_final.append(recer_temp)
            recer_temp = []
    for i in range(len(tex_rec_stats_final)):
        for j in range(1, len(tex_rec_stats_final[i])):
            tex_rec_stats_final[i][j] = float(tex_rec_stats_final[i][j])
    tex_rec_stats_final = pd.DataFrame(tex_rec_stats_final)
    tex_rec_stats_final.columns = ['Player', 'Catches', 'Receiving Yards', 'Receiving TDs', 'Longest Reception']

    full_game_stats = pd.merge(
        tex_pass_stats_final, tex_rush_stats_final, how = "outer", on = "Player")

    full_game_stats = pd.merge(
        full_game_stats, tex_rec_stats_final, how = 'outer', on = "Player")

    full_game_stats = full_game_stats.fillna(0)

    full_game_stats['GameID'] = gameid
    full_game_stats['Date'] = date
    full_game_stats['Home Team'] = home_team
    full_game_stats['Away Team'] = away_team
    full_game_stats['Home Score'] = hscore
    full_game_stats['Away Score'] = ascore
    full_game_stats['Texas Result'] = tex_win
    full_game_stats['Link'] = link

    return full_game_stats

def def_scrape_1(temp_box_soup, year):
    temp = temp_box_soup.find('font', string = "Defensive Statistics")
    temp = temp.find_next('font', string = "Defensive Statistics")
    temp = temp.find_next('pre').text
    temp = temp.replace('Texas Longhorns     ', 'Texas               ')
    temp = temp.replace('TEXAS', 'Texas')

    # Truncate box score for def stats
    def_start = temp.find('Texas               \n## Player               Solo  Ast  Tot  TFL/Yds  FF FR-Yd Intc BrUp Blkd Sack/Yds QH')
    def_temp = temp[def_start:]

    # Get def stats
    def_stats = []    
    line_break = def_temp.find('\n')
    def_temp = def_temp[line_break + 1:]
    line_break = def_temp.find('\n')
    header = '## Last  First  Solo  Ast  Tot  TFL tfl_yds  FF FR fr_yd Int int_yds BrUp Blkd Sack sack_yds QH'.split()
    # def_stats.append(header)
    def_temp = def_temp[line_break:]
    line_break = def_temp.find('\n')
    def_temp = def_temp[line_break + 1:]

    # Now def temp has no header
    line_break = def_temp.find('\n')
    def_temp = def_temp[line_break + 1:]

    while True:
        line_break = def_temp.find('\n')
        line = def_temp[0:line_break + 1]

        if line_break == 0:
            break

        line = line.replace('--', '-')

        game_stat = line.split()

        for i in range(len(game_stat)):
            if game_stat[i] == '.':
                game_stat[i] = ''

        try:
            first_name_catch = re.search(r',\s*(.*)$', game_stat[1]).group(1)
            if first_name_catch:
                last_name_catch = re.search(r'^(.*?),', game_stat[1]).group(1)
                game_stat[1] = last_name_catch
                game_stat.insert(1, first_name_catch)
        except:
            pass

        if len(game_stat) == 13:
            game_stat.insert(2, '')
        
        if len(game_stat) == 15:
            game_stat[2] = str(game_stat[2]) + ' ' + str(game_stat[3])
            game_stat.pop(3)

        if '/' in game_stat[6]:
            parts = game_stat[6].split('/')
            game_stat[6:7] = parts
        else: 
            game_stat[6:7] = ['','']

        if '/' in game_stat[13]:
            parts = game_stat[13].split('/')
            game_stat[13:14] = parts
        else: 
            game_stat[13:14] = ['','']
        
        if '-' in game_stat[9]:
            parts = game_stat[9].split('-')
            game_stat[9:10] = parts
        else: 
            game_stat[9:10] = ['','']

        if '-' in game_stat[11]:
            parts = game_stat[11].split('-')
            game_stat[11:12] = parts
        else: 
            game_stat[11:12] = ['','']

        game_stat[1] = game_stat[1].replace(',', '')
        game_stat[2] = game_stat[2].replace(',', '')

        for i in range(3, 3 + len(game_stat[3:])):
            if game_stat[i]:
                game_stat[i] = float(game_stat[i])
            else:
                game_stat[i] = 0

        def_stats.append(game_stat)
        def_temp = def_temp[line_break + 1:]

    if len(def_stats) > 0:
        if year == '97':
            totals = ['GM', '', 'Game']
        else:
            totals = ['GM', 'Game', '']

        for j in range(3, 3 + len(def_stats[0][3:])):
            total = 0
            for i in range(len(def_stats)):
                total += def_stats[i][j]
            totals.append(total)
        
        
        def_stats.append(totals)

    def_stats = pd.DataFrame(def_stats, columns = header)

    if year == '97':
        def_stats['Last'], def_stats['First'] = def_stats['First'].copy(), def_stats['Last'].copy()
    
    return def_stats

def def_scrape_2(tables, home_team):
    if home_team == 'texas':
        def_table = tables[32]
    else:
        def_table = tables[31]

    def_2d = []
    temp = []
    i = 0
    for row in def_table.find_all('tr')[1:]:
        for val in row.find_all('td'):
            val = val.text.strip()

            if i == 1:
                if val.find(', ') > -1:
                    val = val.split(', ')
                    temp.append(val[0])
                    temp.append(val[1])
                else:
                    temp.append(val)
                    temp.append('')

            elif i == 5 or i == 7 or i == 8 or i == 11:
                if val.find('-') > -1:
                    val = val.split('-')
                    temp.append(val[0])
                    temp.append(val[1])
                elif val.find('/') > -1:
                    val = val.split('/')
                    temp.append(val[0])
                    temp.append(val[1])
                else:
                    temp.append(0)
                    temp.append(0)
            else:
                if val == '.':
                    val = 0
                temp.append(val)
            i += 1

        if temp[0] == '':
            pass
        else:
            for i in range(3, 3 + len(temp[3:])):
                if temp[i]:
                    temp[i] = float(temp[i])
                else:
                    temp[i] = 0
            def_2d.append(temp)
        temp = []
        i = 0

    if len(def_2d) > 0:
            totals = ['GM', 'Game', '']

            for j in range(3, 3 + len(def_2d[0][3:])):
                total = 0
                for i in range(len(def_2d)):
                    total += def_2d[i][j]
                totals.append(total)
            
            def_2d.append(totals)

    def_col = ['##','Last', 'First','Solo','Ast','Tot','TFL', 'tfl_yds','FF','FR', 'fr_yd','Int','int_yds','BrUp','Blkd','Sack','sack_yds','QH']

    def_data = pd.DataFrame(def_2d)
    def_data.columns = def_col

    return def_data

def def_scrape_3(tables, home_team, year_frame):
    if home_team == 'texas':
        tex_def = tables[7]
    else:
        tex_def = tables[6]

    tex_def_stats = tex_def.find_all('td')
    for i in range(len(tex_def_stats)):
        tex_def_stats[i] = tex_def_stats[i].text.strip()
    defender_temp = []
    tex_def_stats_final = []
    for i in range(len(tex_def_stats)):
        temp = tex_def_stats[i]
        temp = temp.replace('--', '-')
        if temp == '-':
            temp = float(0)
        defender_temp.append(temp)
        if len(defender_temp)/16 == 1:
            del defender_temp[12]
            del defender_temp[9]
            del defender_temp[7]
            del defender_temp[5]
            tex_def_stats_final.append(defender_temp)
            defender_temp = []

    tex_def_stats_final = pd.DataFrame(tex_def_stats_final)
    tex_def_stats_final.columns = ['LastFirst','Solo','Ast','Tot','TFL,tfl_yds','Sack,sack_yds','FF','FR,fr_yd','Int,int_yds','BrUp','Blkd','QH']

    if year_frame == '2018' or year_frame == '2019':
        tex_def_stats_final[['Last', 'First']] = tex_def_stats_final['LastFirst'].str.split(' ', n=1, expand=True)
    else:
        tex_def_stats_final[['Last', 'First']] = tex_def_stats_final['LastFirst'].str.split(',', expand=True)
    tex_def_stats_final.drop(columns=['LastFirst'], inplace=True)

    tex_def_stats_final[['TFL', 'tfl_yds']] = tex_def_stats_final['TFL,tfl_yds'].str.split('/', expand=True)
    tex_def_stats_final.drop(columns=['TFL,tfl_yds'], inplace=True)

    tex_def_stats_final[['FR', 'fr_yds']] = tex_def_stats_final['FR,fr_yd'].str.split('/', expand=True)
    tex_def_stats_final.drop(columns=['FR,fr_yd'], inplace=True)

    tex_def_stats_final[['Sack', 'sack_yds']] = tex_def_stats_final['Sack,sack_yds'].str.split('/', expand=True)
    tex_def_stats_final.drop(columns=['Sack,sack_yds'], inplace=True)

    tex_def_stats_final['Int'] = float(0)
    tex_def_stats_final['int_yds'] = float(0)
    for i, value in tex_def_stats_final['Int,int_yds'].items():
        if value != 0:
            int_lst = value.split('-')

            tex_def_stats_final.at[i, 'Int'] = int(int_lst[0])
            tex_def_stats_final.at[i, 'int_yds'] = int(int_lst[1])
        else:
            tex_def_stats_final.at[i, 'Int'] = float(0)
            tex_def_stats_final.at[i, 'int_yds'] = float(0)
    tex_def_stats_final.drop(columns=['Int,int_yds'], inplace=True)

    tex_def_stats_final.replace('-', float(0), inplace=True)

    tex_def_stats_final = tex_def_stats_final[['Last','First','Solo','Ast','Tot','TFL','tfl_yds','Sack','sack_yds','FF','FR','fr_yds','Int','int_yds','BrUp','Blkd','QH']]

    for i in range(2, len(tex_def_stats_final.columns)):
        tex_def_stats_final[tex_def_stats_final.columns[i]] = tex_def_stats_final[tex_def_stats_final.columns[i]].astype(float)

    total_row = tex_def_stats_final.select_dtypes(include=float).sum(axis=0)
    total_row['Last'] = 'Game'
    total_row['First'] = ''
    tex_def_stats_final = pd.concat([tex_def_stats_final, total_row.to_frame().T], ignore_index=True) #tex_def_stats_final.append(total_row, ignore_index=True)

    return tex_def_stats_final

# the only difference btw def scrape 2 and 4 is the tables are 1 off
def def_scrape_4(tables, home_team):
    if home_team == 'texas':
        def_table = tables[33]
    else:
        def_table = tables[32]

    def_2d = []
    temp = []
    i = 0
    for row in def_table.find_all('tr')[1:]:
        for val in row.find_all('td'):
            val = val.text.strip()

            if i == 1:
                if val.find(', ') > -1:
                    val = val.split(', ')
                    temp.append(val[0])
                    temp.append(val[1])
                else:
                    temp.append(val)
                    temp.append('')

            elif i == 5 or i == 7 or i == 8 or i == 11:
                if val.find('-') > -1:
                    val = val.split('-')
                    temp.append(val[0])
                    temp.append(val[1])
                elif val.find('/') > -1:
                    val = val.split('/')
                    temp.append(val[0])
                    temp.append(val[1])
                else:
                    temp.append(0)
                    temp.append(0)
            else:
                if val == '.':
                    val = 0
                temp.append(val)
            i += 1

        if temp[0] == '':
            pass
        else:
            for i in range(3, 3 + len(temp[3:])):
                if temp[i]:
                    temp[i] = float(temp[i])
                else:
                    temp[i] = 0
            def_2d.append(temp)
        temp = []
        i = 0

    if len(def_2d) > 0:
            totals = ['GM', 'Game', '']

            for j in range(3, 3 + len(def_2d[0][3:])):
                total = 0
                for i in range(len(def_2d)):
                    total += def_2d[i][j]
                totals.append(total)
            
            def_2d.append(totals)

    def_col = ['##','Last', 'First','Solo','Ast','Tot','TFL', 'tfl_yds','FF','FR', 'fr_yd','Int','int_yds','BrUp','Blkd','Sack','sack_yds','QH']

    def_data = pd.DataFrame(def_2d)
    def_data.columns = def_col

    return def_data

def def_scrape_5(temp, home_team):
    # Truncate box score for def stats
    def_start = temp.find('## Player               Solo  Ast  Tot  TFL/Yds  FF FR-Yd Intc BrUp Blkd Sack/Yds QH')
    if home_team == 'texas':
        def_temp = temp[def_start + 10:]
        def_start = def_temp.find('## Player               Solo  Ast  Tot  TFL/Yds  FF FR-Yd Intc BrUp Blkd Sack/Yds QH')
        def_temp = def_temp[def_start:]
    else:
        def_temp = temp[def_start:]

    # Get def stats
    def_stats = []    
    line_break = def_temp.find('\n')
    def_temp = def_temp[line_break + 1:]
    line_break = def_temp.find('\n')
    header = '## Last  First  Solo  Ast  Tot  TFL tfl_yds  FF FR fr_yd Int int_yds BrUp Blkd Sack sack_yds QH'.split()
    # def_stats.append(header)
    def_temp = def_temp[line_break:]

    line_break = def_temp.find('\n')
    def_temp = def_temp[line_break + 1:]

    while True:
        line_break = def_temp.find('\n')
        line = def_temp[0:line_break + 1]

        if line_break == 1:
            break

        line = line.replace('--', '-')

        game_stat = line.split()

        for i in range(len(game_stat)):
            if game_stat[i] == '.':
                game_stat[i] = ''

        try:
            first_name_catch = re.search(r',\s*(.*)$', game_stat[1]).group(1)
            if first_name_catch:
                last_name_catch = re.search(r'^(.*?),', game_stat[1]).group(1)
                game_stat[1] = last_name_catch
                game_stat.insert(1, first_name_catch)
        except:
            pass

        if len(game_stat) == 13:
            game_stat.insert(2, '')
        
        if len(game_stat) == 15:
            game_stat[2] = str(game_stat[2]) + ' ' + str(game_stat[3])
            game_stat.pop(3)

        if '/' in game_stat[6]:
            parts = game_stat[6].split('/')
            game_stat[6:7] = parts
        else: 
            game_stat[6:7] = ['','']

        if '/' in game_stat[13]:
            parts = game_stat[13].split('/')
            game_stat[13:14] = parts
        else: 
            game_stat[13:14] = ['','']
        
        if '-' in game_stat[9]:
            parts = game_stat[9].split('-')
            game_stat[9:10] = parts
        else: 
            game_stat[9:10] = ['','']

        if '-' in game_stat[11]:
            parts = game_stat[11].split('-')
            game_stat[11:12] = parts
        else: 
            game_stat[11:12] = ['','']

        game_stat[1] = game_stat[1].replace(',', '')
        game_stat[2] = game_stat[2].replace(',', '')

        for i in range(3, 3 + len(game_stat[3:])):
            if game_stat[i]:
                game_stat[i] = float(game_stat[i])
            else:
                game_stat[i] = 0

        def_stats.append(game_stat)
        def_temp = def_temp[line_break + 1:]

    if len(def_stats) > 0:
        totals = ['GM', 'Game', '']

        for j in range(3, 3 + len(def_stats[0][3:])):
            total = 0
            for i in range(len(def_stats)):
                total += def_stats[i][j]
            totals.append(total)
        
        
        def_stats.append(totals)

    def_stats = pd.DataFrame(def_stats, columns = header)

    return def_stats


Years 1947 - 2007 (excluding the three games in 98) -> stored in master_stats_1

In [68]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
from IPython.display import display, HTML
from tqdm import tqdm
import time
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

# use selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.maximize_window()

# retry mechanism
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
def make_request(url):
    session = requests.Session()
    retry_strategy = Retry(
        total=3,
        status_forcelist=[500, 502, 503, 504],
        allowed_methods=["GET", "POST"],
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    try:
        response = session.get(url, headers = headers)
        response.raise_for_status()
        return response.text
    except:
        return ''

# set up master dataframe
stats = {
    'Player': [],
    'Completions': [],
    'Pass Attempts': [],
    'Interceptions': [],
    'Pass Yards': [],
    'Passing TDs': [],
    'Longest Pass': [],
    'Sacks Taken': [],
    'Rush Attempts': [],
    'Rush Yards Gained': [],
    'Rush Yards Lost': [],
    'Net Rush Yards': [],
    'Rushing TDs': [],
    'Longest Rush': [],
    'Yards Per Rush': [],
    'Catches': [],
    'Receiving Yards': [],
    'Receiving TDs': [],
    'Longest Reception': [],
    'GameID': [],
    'Link': []
}
master_stats_1 = pd.DataFrame(stats)

defs = {'##' : [], 
       'Last' : [], 
       'First' : [], 
       'Solo' : [], 
       'Ast' : [], 
       'Tot' : [], 
       'TFL' : [], 
       'tfl_yds' : [], 
       'FF' : [],
       'FR' : [], 
       'fr_yd' : [], 
       'Int' : [], 
       'int_yds' : [], 
       'BrUp' : [], 
       'Blkd' : [], 
       'Sack' : [], 
       'sack_yds' : [],
       'QH' : []
}
master_def_1 = pd.DataFrame(defs)

games = {
    'Home Team' : [],
    'Away Team'	: [],
    'Home Score' : [],
    'Away Score' : [],
    'Texas Result' : [],
    'Box Score' : []
}
master_games_1 = pd.DataFrame(games)

missed_games_1 = []

# get links for each season
years_list = [str(47), str(48)]
for i in range(50, 100):
    years_list.append(str(i))
for i in range(0, 8):
    years_list.append(f"{i:02d}")
# years_list.append('08')

season_links = []
for i in years_list:
    season_link = 'https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/' + i + '/teamstat.htm'
    season_links.append(season_link)

for x in tqdm(range(len(season_links)), desc = "Database building..."):
    season = season_links[x] # paste specific season link here when troubleshooting
    year = years_list[x] # change year manually to when troubleshooting
    box_score_links = []

    # get season
    if int(year) >= 47:
        year_frame = '19' + year
    else:
        year_frame = '20' + year

    if year == '98':
        box_score_links = ['https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/98/ut-ucla.htm',
                            'https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/98/ut-msu.htm',
                            'https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/98/ut-ksu.htm',
                            'https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/98/ut-ou.htm',
                            'https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/98/ut-bu.htm',
                            'https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/98/ut-nu.htm',
                            'https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/98/ut-osu.htm',
                            'https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/98/ut-ttu.htm',
                            'https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/98/ut-tam.htm']
    # elif year == '08':
    #     # 2008 links
    #     box_score_links = ['http://stats.texassports.com/sports/m-footbl/2008-2009/ut2.html', 
    #                         'http://stats.texassports.com/sports/m-footbl/2008-2009/ut3.html', 
    #                         'http://stats.texassports.com/sports/m-footbl/2008-2009/ut4.html', 
    #                         'http://stats.texassports.com/sports/m-footbl/2008-2009/ut5.html', 
    #                         'http://stats.texassports.com/sports/m-footbl/2008-2009/ut6.html', 
    #                         'http://stats.texassports.com/sports/m-footbl/2008-2009/ut7.html', 
    #                         'http://stats.texassports.com/sports/m-footbl/2008-2009/ut8.html', 
    #                         'http://stats.texassports.com/sports/m-footbl/2008-2009/ut9.html', 
    #                         'http://stats.texassports.com/sports/m-footbl/2008-2009/ut10.html', 
    #                         'http://stats.texassports.com/sports/m-footbl/2008-2009/ut11.html', 
    #                         'http://stats.texassports.com/sports/m-footbl/2008-2009/ut12.html', 
    #                         'http://stats.texassports.com/sports/m-footbl/2008-2009/ut13.html']
    else:
        # open season page
        driver.get(season)
        texas_sports_soup = BeautifulSoup(driver.page_source)
        table = texas_sports_soup.find("table")
        rows = table.tbody.find_all('tr')[1:]
        for row in rows:
            box_score = row.find_all('td')[-1]
            try:
                link_tail_temp = box_score.font.a['href']
                if link_tail_temp == '../../index1919.html':
                    missed_games_1.append(row.find_all('td')[-2].get_text())
                else:
                    built_link = 'https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/' + year + '/' + link_tail_temp
                    box_score_links.append(built_link)
            except:
                pass

            # must catch the random 90s TAMU games that are mislinked
            if year in [str(91), str(92), str(93), str(95), str(96)]:
                box_score_links.append('https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/' + year + '/' + 'UT-A&M.HTM')

    for link in box_score_links:
        # print(link_tail, year)        
        # Get full page soup
        # link = 'https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/92/ut_nt.htm' # troubleshooting
        response = make_request(link)
        temp_box_soup = BeautifulSoup(response, 'html.parser')

        if response == '':
            driver.get(link)
            # driver.get('https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/06/ut11.htm') # for troubleshooting
            temp_box_soup = BeautifulSoup(driver.page_source)
        temp_text = temp_box_soup.text
        temp_text = temp_text.replace("Texas Longhorns", "Texas")
        temp_text = temp_text.replace('TEXAS', 'Texas')

        if year == '06':
            # Get away team
            away_index = 0
            away_endex = temp_text.find(' vs ')
            away_team = temp_text[away_index: away_endex].strip().lower()

            # Get home team
            home_index = temp_text.find(' vs ')
            home_endex = temp_text.find(' (')
            home_team = temp_text[home_index + 4: home_endex].strip().lower()

            # Get game date
            date_index = temp_text.find('(')
            date_endex = temp_text.find(")")
            date = temp_text[date_index + 1: date_endex].strip()
            date = date.replace(",", "")
            date = date[:-4] + ',' + date[-4:]
            date = date.replace("Sept", "Sep")
            date = date.replace(" ", "")
            try:
                date = datetime.strptime(date, "%b%d,%Y")
            except ValueError:
                date = datetime.strptime(date, "%b.%d,%Y")
            
            # get away score
            temp_index = temp_text.find('Score by Quarters     1  2  3  4   Score')
            temp_text_new = temp_text[temp_index:]
            temp_index = temp_text_new.find('\n')
            temp_text_new = temp_text_new[temp_index + 1:]
            temp_index = temp_text_new.find('\n')
            temp_text_new = temp_text_new[temp_index + 1:]
            ascore_index = temp_text_new.find(' - ') + 3
            ascore_endex = temp_text_new.find('Record: ')
            ascore = temp_text_new[ascore_index:ascore_endex].strip()
            ascore = float(ascore)

            # get home score
            temp_text_new = temp_text_new[ascore_endex + 1:]
            hscore_index = temp_text_new.find(' - ') + 3
            hscore_endex = temp_text_new.find('Record: ')
            hscore = temp_text_new[hscore_index:hscore_endex].strip()
            hscore = float(hscore)
            
        else:
            # Get game date
            date_index = temp_text.find('Date: ')
            date_endex = temp_text.find("Site: ")
            date = temp_text[date_index + 6: date_endex].strip()
            if date == "0ct 10, 1959":
                date = "Oct 10, 1959"
            date = date.replace(",", "")
            date = date[:-4] + ',' + date[-4:]
            date = date.replace("Sept", "Sep")
            date = date.replace(" ", "")
            try:
                date = datetime.strptime(date, "%b%d,%Y")
            except ValueError:
                date = datetime.strptime(date, "%b.%d,%Y")      

            # Get away team
            away_index = 0
            away_endex = temp_text.find(' vs ')
            away_team = temp_text[away_index: away_endex].strip().lower()

            # Get home team
            home_index = temp_text.find(' vs ')
            home_endex = temp_text.find(' (')
            home_team = temp_text[home_index + 4: home_endex].strip().lower()

            # get away score
            temp_index = temp_text.find('Score by Quarters     1  2  3  4   Score')
            temp_text_new = temp_text[temp_index:]
            temp_index = temp_text_new.find('\n')
            temp_text_new = temp_text_new[temp_index + 1:]
            temp_index = temp_text_new.find('\n')
            temp_text_new = temp_text_new[temp_index + 1:]
            ascore_index = temp_text_new.find(' - ') + 3
            ascore_endex = temp_text_new.find('\n')
            ascore = temp_text_new[ascore_index:ascore_endex].strip()
            ascore = float(ascore)

            # get home score
            temp_text_new = temp_text_new[ascore_endex + 1:]
            hscore_index = temp_text_new.find(' - ') + 3
            hscore_endex = temp_text_new.find('\n')
            hscore = temp_text_new[hscore_index:hscore_endex].strip()
            hscore = float(hscore)

        # did texas win?
        if (home_team == 'texas' and hscore > ascore) or (away_team == 'texas' and hscore < ascore):
            tex_win = "Win"
        elif (home_team == 'texas' and hscore < ascore) or (away_team == 'texas' and hscore > ascore):
            tex_win = "Loss"
        else:
            tex_win = "Tie"

        # get UT box score text
        temp = temp_box_soup.find('font', string = "Individual Statistics")
        temp = temp.find_next('font', string = "Individual Statistics")
        temp = temp.find_next('pre').text
        temp = temp.replace('Texas Longhorns', 'Texas')
        temp = temp.replace('TEXAS', 'Texas')
        start = temp.find('\nTexas\n')
        temp = temp[start:]
        end = temp.find('Punting               No.  Yds   Avg Long In20')
        temp = temp[:end]

        # Truncate box score for rushing stats
        rush_start = temp.find('Rushing              No Gain Loss  Net TD Lg  Avg')
        rush_end = temp.find('Passing              ') - 2
        rush_temp = temp[rush_start:rush_end]

        # Get rushing stats
        rush_stats = []    
        line_break = rush_temp.find('\n')
        header = rush_temp[0:line_break].split()
        rush_stats.append(header)
        rush_temp = rush_temp[line_break:]
        line_break = rush_temp.find('\n')
        rush_temp = rush_temp[line_break + 1:]

        # Now rush temp has no header
        line_break = rush_temp.find('\n')
        rush_temp = rush_temp[line_break + 1:]
        while True:
            line_break = rush_temp.find('\n')
            line = rush_temp[0:line_break + 1]
            game_stat = line.split()
            if rush_temp.find('\n') == -1:
                line = rush_temp
                game_stat = line.split()          
                rush_stats.append(game_stat)
                rush_temp = rush_temp[line_break + 1:]
                break
            else:
                # combines first n name columns
                while len(game_stat) > 8: 
                    name = game_stat[0] + ' ' + game_stat[1]
                    stats = game_stat[2:]
                    game_stat = [name] + stats
                rush_stats.append(game_stat)
                rush_temp = rush_temp[line_break + 1:]
        
        # Truncate box score for passing stats
        pass_start = temp.find('Passing              ')
        pass_end = temp.find('Receiving             No.  Yds   TD Long') - 2
        pass_temp = temp[pass_start:pass_end]

        # Get passing stats
        pass_stats = []    
        line_break = pass_temp.find('\n')
        header = pass_temp[0:line_break].split()
        pass_stats.append(header)
        pass_temp = pass_temp[line_break:]
        line_break = pass_temp.find('\n')
        pass_temp = pass_temp[line_break + 1:]

        # Now pass temp has no header
        line_break = pass_temp.find('\n')
        pass_temp = pass_temp[line_break + 1:]
        while True:
            line_break = pass_temp.find('\n')
            line = pass_temp[0:line_break + 1]
            game_stat = line.split()
            if pass_temp.find('\n') == -1:
                line = pass_temp
                game_stat = line.split()
                pass_stats.append(game_stat)
                pass_temp = pass_temp[line_break + 1:]
                break
            else:
                # combines first n name columns
                while len(game_stat) > 6: 
                    name = game_stat[0] + ' ' + game_stat[1]
                    stats = game_stat[2:]
                    game_stat = [name] + stats
                pass_stats.append(game_stat)
                pass_temp = pass_temp[line_break + 1:]
        
        # Truncate box score for rec stats
        rec_start = temp.find('Receiving             No.  Yds   TD Long')
        rec_temp = temp[rec_start:-2]

        # Get rec stats
        rec_stats = []    
        line_break = rec_temp.find('\n')
        header = rec_temp[0:line_break].split()
        rec_stats.append(header)
        rec_temp = rec_temp[line_break:]
        line_break = rec_temp.find('\n')
        rec_temp = rec_temp[line_break + 1:]

        # Now rec temp has no header
        line_break = rec_temp.find('\n')
        rec_temp = rec_temp[line_break + 1:]
        while True:
            line_break = rec_temp.find('\n')
            line = rec_temp[0:line_break + 1]
            game_stat = line.split()
            if rec_temp.find('\n') == -1:
                line = rec_temp
                game_stat = line.split()
                rec_stats.append(game_stat)
                rec_temp = rec_temp[line_break + 1:]
                break
            else:
                # combines first n name columns
                while len(game_stat) > 5: 
                    name = game_stat[0] + ' ' + game_stat[1]
                    stats = game_stat[2:]
                    game_stat = [name] + stats
                rec_stats.append(game_stat)
                rec_temp = rec_temp[line_break + 1:]

        ##############################################################
        # Now that we have the stats in 2d lists, we need to make sure they aren't just strings

        # First, we must address the formatting of the passing cmp-att-int format
        for row in pass_stats:
            new_element = row.pop(1).split('-')
            row[1:1] = new_element
        
        # Next, we must make sure the elements are floats and not strings
        frames = [pass_stats, rush_stats, rec_stats]
        for frame in frames:
            for i in range(1,len(frame)):
                for j in range(1,len(frame[i])):
                    frame[i][j] = float(frame[i][j])
                    
        # Now, we make the arrays into dataframes using panda
        # admittedly i shouldve done this earlier but oh well
        rush_data = pd.DataFrame(rush_stats[1:])
        rush_data.columns = ['Player', 'Rush Attempts', 'Rush Yards Gained', 'Rush Yards Lost', 'Net Rush Yards', 'Rushing TDs', 'Longest Rush', 'Yards Per Rush']

        pass_data = pd.DataFrame(pass_stats[1:])
        if 40 < float(year) < 89:
            pass_data.columns = ['Player', 'Completions', 'Pass Attempts', 'Interceptions', 'Pass Yards', 'Passing TDs', 'Longest Pass', 'Sacks Taken']
        else:
            pass_data.columns = ['Player', 'Pass Attempts', 'Completions', 'Interceptions', 'Pass Yards', 'Passing TDs', 'Longest Pass', 'Sacks Taken']


        rec_data = pd.DataFrame(rec_stats[1:])
        rec_data.columns = ['Player', 'Catches', 'Receiving Yards', 'Receiving TDs', 'Longest Reception']
        
        # Finally, time to merge the data into one full dataframe for the full game
        full_game_data = pd.merge(
            pass_data, rush_data, how = "outer", on = "Player"
        )
        full_game_data = pd.merge(
            full_game_data, rec_data, how = 'outer', on = "Player"
        )
        full_game_data = full_game_data.fillna(0)

        # small thing but i want to take the ellipsis out of the totals category
        full_game_data = full_game_data.replace('Totals...', 'Total')

        # now make the gameid
        gameid = away_team.replace(" ", "").lower() + '_' + home_team.replace(" ", "").lower() + '_' + str(date.month) + '_' + str(date.day) + '_' + str(date.year)
        gameid_list = [gameid]

        '''
        game_df = {'Home Team' : [home_team],
                'Away Team' : [away_team],
                'Home Score' : [hscore],
                'Away Score' : [ascore],
                'Texas Result' : [tex_win],
                'Box Score' : [full_game_data]
                }
        
        game_df = pd.DataFrame(game_df, index = gameid_list)

        # finally append it to the master games
        master_games = pd.concat([master_games, game_df], ignore_index = True)

        # empty out game_df
        game_df = pd.DataFrame()
        '''

        def_stats = def_scrape_1(temp_box_soup, year)

        ##############################################################
        # the last thing I want to do is to create one large dataframe with every single game performance ever
        # this will contain duplicate players for their different performances in different games
        # much less concise, much more usefull (probably)
        # first add gameid column
        full_game_data['GameID'] = gameid
        full_game_data['Date'] = date
        full_game_data['Home Team'] = home_team
        full_game_data['Away Team'] = away_team
        full_game_data['Home Score'] = hscore
        full_game_data['Away Score'] = ascore
        full_game_data['Texas Result'] = tex_win
        full_game_data['Link'] = link
        full_game_data['Season'] = year_frame

        def_stats['GameID'] = gameid
        def_stats['Date'] = date
        def_stats['Home Team'] = home_team
        def_stats['Away Team'] = away_team
        def_stats['Home Score'] = hscore
        def_stats['Away Score'] = ascore
        def_stats['Texas Result'] = tex_win
        def_stats['Link'] = link
        def_stats['Season'] = year_frame
        
        # now add it to the master stats
        master_stats_1 = pd.concat([master_stats_1, full_game_data], ignore_index = True)
        master_def_1 = pd.concat([master_def_1, def_stats], ignore_index = True)

        # finally empty out the full game dataframe
        full_game_data = pd.DataFrame() 
        
# print(master_stats_1)
master_stats_1.to_csv('master_stats_1.csv', index = False)
master_def_1.to_csv('master_def_1.csv', index = False)
# master_games.to_csv('master_games.csv', index = False)

driver.quit()

Database building...: 100%|██████████| 60/60 [08:07<00:00,  8.13s/it]


get stats from blue table style (3 games in 98 and 2009 -2014) 
note: i have skipped 2008 for the moment, also missing vs arkansas 2014 and vs kansas 2009 vs nebraska 2009
-> stored in master_stats_2

In [53]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
from IPython.display import display, HTML
from tqdm import tqdm
import time
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

# use selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.maximize_window()

# retry mechanism
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
def make_request(url):
    session = requests.Session()
    retry_strategy = Retry(
        total=3,
        status_forcelist=[500, 502, 503, 504],
        allowed_methods=["GET", "POST"],
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    try:
        response = session.get(url, headers = headers)
        response.raise_for_status()
        return response.text
    except:
        return ''

# set up master dataframe
stats = {
    'Player': [],
    'Completions': [],
    'Pass Attempts': [],
    'Interceptions': [],
    'Pass Yards': [],
    'Passing TDs': [],
    'Longest Pass': [],
    'Sacks Taken': [],
    'Rush Attempts': [],
    'Rush Yards Gained': [],
    'Rush Yards Lost': [],
    'Net Rush Yards': [],
    'Rushing TDs': [],
    'Longest Rush': [],
    'Yards Per Rush': [],
    'Catches': [],
    'Receiving Yards': [],
    'Receiving TDs': [],
    'Longest Reception': [],
    'GameID': [],
    'Link': []
}
master_stats_2 = pd.DataFrame(stats)

defs = {'##' : [], 
       'Last' : [], 
       'First' : [], 
       'Solo' : [], 
       'Ast' : [], 
       'Tot' : [], 
       'TFL' : [], 
       'tfl_yds' : [], 
       'FF' : [],
       'FR' : [], 
       'fr_yd' : [], 
       'Int' : [], 
       'int_yds' : [], 
       'BrUp' : [], 
       'Blkd' : [], 
       'Sack' : [], 
       'sack_yds' : [],
       'QH' : []
}
master_def_2 = pd.DataFrame(defs)

games = {
    'Home Team' : [],
    'Away Team'	: [],
    'Home Score' : [],
    'Away Score' : [],
    'Texas Result' : [],
    'Box Score' : []
}
master_games_2 = pd.DataFrame(games)

missed_games_2 = []

driver.get('https://texassports.com/sports/2013/7/21/FB_0721134841.aspx?id=131')
texas_sports_soup = BeautifulSoup(driver.page_source)
table_list = texas_sports_soup.find_all("table")

game_links = []
# 1998 links
rows = table_list[24].tbody.find_all('tr')[2:]
for row in rows:
    box_score = row.find_all('td')[-1]
    try:
        link_temp = box_score.a['href']
        game_links.append(link_temp)
    except:
        pass

game_links = [['https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/98/ut-isu.htm', '1998'],
              ['https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/98/ut-ru.htm', '1998'],
              ['https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/98/ut-nmsu.htm', '1998']]

driver.get('https://texassports.com/sports/2013/7/21/FB_0721134841.aspx?id=131')
texas_sports_soup = BeautifulSoup(driver.page_source)
table_list = texas_sports_soup.find_all("table")

# get most recent year
temp = table_list[0].find('td').get_text()
year_index = temp.find('\n')
year = temp[year_index - 4:year_index]

years_past_2022 = int(year) - 2022

game_links = []

# links for 2009-2014
for table in table_list[8 + years_past_2022 : 14 + years_past_2022]:
    row_ind = 3

    # get year
    temp = table.find('td').get_text()
    year_index = temp.find('\n')
    year = temp[year_index - 4:year_index]

    if year == '':
        # get year
        temp = table.find_all('td')[1].get_text()
        year_index = temp.find('\n')
        year = temp[year_index - 4:year_index]
        row_ind += 1

    # get box score links
    rows = table.tbody.find_all('tr')[row_ind:]
    for row in rows:
        box_score = row.find_all('td')[-1]
        try:
            link_temp = box_score.find_all('a')[1]['href']
            game_links.append(list((link_temp, year)))
        except:
            pass

game_links.remove(['http://texassports.com/boxscore.aspx?path=football&id=8599', '2014'])
game_links.remove(['http://stats.texassports.com/sports/m-footbl/2009-2010/ut11.html', '2009'])
game_links.remove(['http://stats.texassports.com/sports/m-footbl/2009-2010/big12fb.html', '2009'])

for i in tqdm(range(len(game_links)), desc = "Database building..."):
# for i in range(len(game_links)):
    # Get full page soup
    link = game_links[i][0]
    year = game_links[i][1]

    response = make_request(link)
    temp_box_soup = BeautifulSoup(response, 'html.parser')

    if response == '':
        driver.get(link)
        # driver.get('https://stats.texassports.com/custompages/sports/m-footbl/2009-2010/ut11.html') # for troubleshooting
        temp_box_soup = BeautifulSoup(driver.page_source)
    
    temp_text = temp_box_soup.text
    temp_text = temp_text.replace("Texas Longhorns", "Texas")
    temp_text = temp_text.replace('TEXAS', 'Texas')

    # Get game date
    date_index = temp_text.find('Date: ')
    date_endex = temp_text.find("Site: ")
    date = temp_text[date_index + 6: date_endex - 3].strip()
    date = date.replace(",", "")
    date = date[:-4] + ',' + date[-4:]
    date = date.replace("Sept", "Sep")
    date = date.replace(" ", "")
    try:
        date = datetime.strptime(date, "%b%d,%Y")
    except ValueError:
        date = datetime.strptime(date, "%b.%d,%Y")      

    tables = temp_box_soup.find_all("table")
    score_table = tables[2]
    away_t_row = 1
    home_t_row = 2
    
    away_t_col = 0
    home_t_col = 0

    away_s_col = 5
    home_s_col = 5

    home_team = score_table.find_all('tr')[home_t_row].find_all('td')[home_t_col].text.strip().lower()
    away_team = score_table.find_all('tr')[away_t_row].find_all('td')[away_t_col].text.strip().lower()
    ascore = float(score_table.find_all('tr')[away_t_row].find_all('td')[away_s_col].text.strip())
    hscore = float(score_table.find_all('tr')[home_t_row].find_all('td')[home_s_col].text.strip())

    # did texas win?
    if (home_team == 'texas' and hscore > ascore) or (away_team == 'texas' and hscore < ascore):
        tex_win = "Win"
    elif (home_team == 'texas' and hscore < ascore) or (away_team == 'texas' and hscore > ascore):
        tex_win = "Loss"
    else:
        tex_win = "Tie"     

    if home_team == 'texas':
        rush_table = tables[10]
        pass_table = tables[12]
        rec_table = tables[14]
    else:
        rush_table = tables[9]
        pass_table = tables[11]
        rec_table = tables[13]
    
    # rush stats
    rush_2d = []
    temp = []
    for row in rush_table.find_all('tr')[1:]:
        for val in row.find_all('td'):
            temp.append(val.text.strip())
        if temp[0] == '':
            pass
        else:
            rush_2d.append(temp)
        temp = []

    rush_col = ['Player', 'Rush Attempts', 'Rush Yards Gained', 'Rush Yards Lost', 'Net Rush Yards', 'Rushing TDs', 'Longest Rush', 'Yards Per Rush']

    rush_data = pd.DataFrame(rush_2d)
    rush_data.columns = rush_col

    # pass stats
    pass_2d = []
    temp = []
    for row in pass_table.find_all('tr')[1:]:
        for val in row.find_all('td'):
            temp.append(val.text.strip())
        if temp[0] == '':
            pass
        else:
            pass_2d.append(temp)
        temp = []

    for row in pass_2d:
        new_element = row.pop(1).split('-')
        row[1:1] = new_element

    pass_col = ['Player', 'Completions', 'Pass Attempts', 'Interceptions', 'Pass Yards', 'Passing TDs', 'Longest Pass', 'Sacks Taken']

    pass_data = pd.DataFrame(pass_2d)
    pass_data.columns = pass_col

    # pass stats
    rec_2d = []
    temp = []
    for row in rec_table.find_all('tr')[1:]:
        for val in row.find_all('td'):
            temp.append(val.text.strip())
        if temp[0] == '':
            pass
        else:
            rec_2d.append(temp)
        temp = []

    rec_col = ['Player', 'Catches', 'Receiving Yards', 'Receiving TDs', 'Longest Reception']
            
    frames = [pass_2d, rush_2d, rec_2d]
    for frame in frames:
        for i in range(0,len(frame)):
            for j in range(1,len(frame[i])):
                frame[i][j] = float(frame[i][j])

    rec_data = pd.DataFrame(rec_2d)
    rec_data.columns = rec_col

    full_game_data = pd.merge(pass_data, rush_data, how = "outer", on = "Player")
    full_game_data = pd.merge(full_game_data, rec_data, how = 'outer', on = "Player")
    full_game_data = full_game_data.fillna(0)

    full_game_data = full_game_data.replace('Totals...', 'Total')

    gameid = away_team.replace(" ", "").lower() + '_' + home_team.replace(" ", "").lower() + '_' + str(date.month) + '_' + str(date.day) + '_' + str(date.year)
    gameid_list = [gameid]

    def_stats = def_scrape_2(tables, home_team)

    full_game_data['GameID'] = gameid
    full_game_data['Date'] = date
    full_game_data['Home Team'] = home_team
    full_game_data['Away Team'] = away_team
    full_game_data['Home Score'] = hscore
    full_game_data['Away Score'] = ascore
    full_game_data['Texas Result'] = tex_win
    full_game_data['Link'] = link
    full_game_data['Season'] = year

    def_stats['GameID'] = gameid
    def_stats['Date'] = date
    def_stats['Home Team'] = home_team
    def_stats['Away Team'] = away_team
    def_stats['Home Score'] = hscore
    def_stats['Away Score'] = ascore
    def_stats['Texas Result'] = tex_win
    def_stats['Link'] = link
    def_stats['Season'] = year

    master_stats_2 = pd.concat([master_stats_2, full_game_data], ignore_index = True)

    master_def_2 = pd.concat([master_def_2, def_stats], ignore_index = True)

    # finally empty out the full game dataframe
    full_game_data = pd.DataFrame() 

master_stats_2.to_csv('master_stats_2.csv', index = False)
master_def_2.to_csv('master_def_2.csv', index = False)
# master_games.to_csv('master_games.csv', index = False)

driver.quit()

Database building...: 100%|██████████| 69/69 [01:45<00:00,  1.52s/it]


Get Links from UT master results (list for post 2015)

In [41]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
from IPython.display import display, HTML
from tqdm import tqdm
import time

# use selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.maximize_window()


driver.get('https://texassports.com/sports/2013/7/21/FB_0721134841.aspx?id=131')
texas_sports_soup = BeautifulSoup(driver.page_source)
driver.quit()
table_list = texas_sports_soup.find_all("table")

# get most recent year
temp = table_list[0].find('td').get_text()
year_index = temp.find('\n')
year = temp[year_index - 4:year_index]

years_past_2022 = int(year) - 2022

# link dictionary
link_dict = {}
for table in table_list[:8 + years_past_2022]:
    # get year
    temp = table.find('td').get_text()
    year_index = temp.find('\n')
    year = temp[year_index - 4:year_index]

    # get box score links
    rows = table.tbody.find_all('tr')[2:]
    box_score_links = []
    for row in rows:
        box_score = row.find_all('td')[-1]
        try:
            link_temp = box_score.a['href']
            box_score_links.append(link_temp)
        except:
            pass
    
    link_dict[year] = box_score_links

ut website scrape (2015-2022) missing la tech 2019 (plus http://texassports.com/boxscore.aspx?path=football&id=8599)

In [96]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
from IPython.display import display, HTML
from tqdm import tqdm
import time
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

# use selenium
options = webdriver.ChromeOptions()
options.add_argument("--headless=new")
options.add_argument("--ignore-certificate-errors")
options.add_argument("--incognito")
service = Service()
driver = webdriver.Chrome(service=service, options=options)
driver.maximize_window()

# retry mechanism
headers = {'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/58.0.3029.110 Safari/537.3'}
def make_request(url):
    session = requests.Session()
    retry_strategy = Retry(
        total=3,
        status_forcelist=[500, 502, 503, 504],
        allowed_methods=["GET", "POST"],
    )
    adapter = HTTPAdapter(max_retries=retry_strategy)
    session.mount("http://", adapter)
    session.mount("https://", adapter)

    try:
        response = session.get(url, headers = headers)
        response.raise_for_status()
        return response.text
    except:
        return ''

driver.get('https://texassports.com/sports/2013/7/21/FB_0721134841.aspx?id=131')
texas_sports_soup = BeautifulSoup(driver.page_source)
table_list = texas_sports_soup.find_all("table")

# set up master dataframe
stats = {
    'Player': [],
    'Completions': [],
    'Pass Attempts': [],
    'Interceptions': [],
    'Pass Yards': [],
    'Passing TDs': [],
    'Longest Pass': [],
    'Sacks Taken': [],
    'Rush Attempts': [],
    'Rush Yards Gained': [],
    'Rush Yards Lost': [],
    'Net Rush Yards': [],
    'Rushing TDs': [],
    'Longest Rush': [],
    'Yards Per Rush': [],
    'Catches': [],
    'Receiving Yards': [],
    'Receiving TDs': [],
    'Longest Reception': [],
    'GameID': [],
    'Link': []
}
master_stats_3 = pd.DataFrame(stats)

defs = {'Last' : [], 
       'First' : [], 
       'Solo' : [], 
       'Ast' : [], 
       'Tot' : [], 
       'TFL' : [], 
       'tfl_yds' : [], 
       'FF' : [],
       'FR' : [], 
       'fr_yds' : [], 
       'Int' : [], 
       'int_yds' : [], 
       'BrUp' : [], 
       'Blkd' : [], 
       'Sack' : [], 
       'sack_yds' : [],
       'QH' : []
}
master_def_3 = pd.DataFrame(defs)

# link list
links_2015_2022 = []

# get most recent year
temp = table_list[0].find('td').get_text()
year_index = temp.find('\n')
year = temp[year_index - 4:year_index]

years_past_2022 = int(year) - 2022

for table in table_list[:8 + years_past_2022]:
    # get year
    temp = table.find('td').get_text()
    year_index = temp.find('\n')
    year = temp[year_index - 4:year_index]

    # get box score links
    rows = table.tbody.find_all('tr')[2:]
    for row in rows:
        box_score = row.find_all('td')[-1]
        try:
            link_temp = box_score.a['href']
            links_2015_2022.append([link_temp, year])
        except:
            pass

# deal with mislinked la tech 2019
links_2015_2022[links_2015_2022.index(['hthttps://texassports.com/boxscore.aspx?path=football&id=12601', '2019'])] = ['https://texassports.com/boxscore.aspx?path=football&id=12601', '2019']

# texas vs arkansas 2014
links_2015_2022.append(['http://texassports.com/boxscore.aspx?path=football&id=8599', '2014'])

for i in tqdm(range(len(links_2015_2022)), desc = "Database building..."):
# for i in range(len(links_2015_2022)):
    url = links_2015_2022[i][0]
    year_frame = links_2015_2022[i][1]
    # driver.get(url)
    # driver.implicitly_wait(2) # wait a bit
    # page_source = driver.page_source
    # soup = BeautifulSoup(page_source)

    response = make_request(url)
    soup = BeautifulSoup(response, 'html.parser')

    if response == '':
        # print(url)
        driver.get(url)
        # driver.implicitly_wait(2) # wait a bit
        page_source = driver.page_source
        soup = BeautifulSoup(page_source)

    individual_stats = soup.find('section', id='individual-stats')
    tables = individual_stats.find_all('table')

    score_table = soup.find('table')
    score_table = score_table.find_all('td')
    for i in range(1, len(score_table)):
        try:
            x = int(score_table[i+1].text)
        except:
            ascore = float(score_table[i].text)
            home_team = score_table[i+1].find_all('span', class_='hide-on-small-down')[0].get_text().strip().lower()
            break

    away_team = score_table[0].find_all('span', class_='hide-on-small-down')[0].get_text().strip().lower()
    hscore = float(score_table[-1].text)

    home_team = home_team.replace("Winner", "")
    away_team = away_team.replace("Winner", "")

    if (home_team == 'texas' and hscore > ascore) or (away_team == 'texas' and hscore < ascore):
        tex_win = "Win"
    elif (home_team == 'texas' and hscore < ascore) or (away_team == 'texas' and hscore > ascore):
        tex_win = "Loss"
    else:
        tex_win = "Tie"

    # get date
    big_html = soup.text
    date_index = big_html.find('Date:')
    date_endex = big_html.find('Site:')
    date = big_html[date_index + 6: date_endex].strip()
    date = datetime.strptime(date, "%m/%d/%Y")

    # make gameid
    gameid = away_team.replace(" ", "").lower() + '_' + home_team.replace(" ", "").lower() + '_' + str(date.month) + '_' + str(date.day) + '_' + str(date.year)

    if home_team == 'texas':
        tex_pass = tables[1]
        tex_rush = tables[3]
        tex_rec = tables[5]
    else:
        tex_pass = tables[0]
        tex_rush = tables[2]
        tex_rec = tables[4]

    tex_pass_stats = tex_pass.find_all('td')
    for i in range(len(tex_pass_stats)):  # convert passers to text
        tex_pass_stats[i] = tex_pass_stats[i].text.strip()
    passer_temp = []
    tex_pass_stats_final = []
    for i in range(len(tex_pass_stats)):
        passer_temp.append(tex_pass_stats[i])
        if len(passer_temp)/8 == 1:
            tex_pass_stats_final.append(passer_temp)
            passer_temp = []
    for i in range(len(tex_pass_stats_final)):
        for j in range(1, len(tex_pass_stats_final[i])):
            tex_pass_stats_final[i][j] = float(tex_pass_stats_final[i][j])
    tex_pass_stats_final = pd.DataFrame(tex_pass_stats_final)
    tex_pass_stats_final.columns = ['Player', 'Completions', 'Pass Attempts', 'Pass Yards', 'Passing TDs', 'Interceptions', 'Longest Pass', 'Sacks Taken']

    tex_rush_stats = tex_rush.find_all('td')
    for i in range(len(tex_rush_stats)):  # convert passers to text
        tex_rush_stats[i] = tex_rush_stats[i].text.strip()
    rusher_temp = []
    tex_rush_stats_final = []
    for i in range(len(tex_rush_stats)):
        rusher_temp.append(tex_rush_stats[i])
        if len(rusher_temp)/8 == 1:
            tex_rush_stats_final.append(rusher_temp)
            rusher_temp = []
    for i in range(len(tex_rush_stats_final)):
        for j in range(1, len(tex_rush_stats_final[i])):
            tex_rush_stats_final[i][j] = float(tex_rush_stats_final[i][j])
    tex_rush_stats_final = pd.DataFrame(tex_rush_stats_final)
    tex_rush_stats_final.columns = ['Player', 'Rush Attempts', 'Rush Yards Gained', 'Rush Yards Lost', 'Net Rush Yards', 'Rushing TDs', 'Longest Rush', 'Yards Per Rush']

    tex_rec_stats = tex_rec.find_all('td')
    for i in range(len(tex_rec_stats)):  # convert passers to text
        tex_rec_stats[i] = tex_rec_stats[i].text.strip()
    recer_temp = []
    tex_rec_stats_final = []
    for i in range(len(tex_rec_stats)):
        recer_temp.append(tex_rec_stats[i])
        if len(recer_temp)/5 == 1:
            tex_rec_stats_final.append(recer_temp)
            recer_temp = []
    for i in range(len(tex_rec_stats_final)):
        for j in range(1, len(tex_rec_stats_final[i])):
            tex_rec_stats_final[i][j] = float(tex_rec_stats_final[i][j])
    tex_rec_stats_final = pd.DataFrame(tex_rec_stats_final)
    tex_rec_stats_final.columns = ['Player', 'Catches', 'Receiving Yards', 'Receiving TDs', 'Longest Reception']

    full_game_stats = pd.merge(
        tex_pass_stats_final, tex_rush_stats_final, how = "outer", on = "Player")

    full_game_stats = pd.merge(
        full_game_stats, tex_rec_stats_final, how = 'outer', on = "Player")

    full_game_stats = full_game_stats.fillna(0)

    full_def_stats = def_scrape_3(tables, home_team, year_frame)

    full_game_stats['GameID'] = gameid
    full_game_stats['Date'] = date
    full_game_stats['Home Team'] = home_team
    full_game_stats['Away Team'] = away_team
    full_game_stats['Home Score'] = hscore
    full_game_stats['Away Score'] = ascore
    full_game_stats['Texas Result'] = tex_win
    full_game_stats['Link'] = url
    full_game_stats['Season'] = year_frame

    full_def_stats['GameID'] = gameid
    full_def_stats['Date'] = date
    full_def_stats['Home Team'] = home_team
    full_def_stats['Away Team'] = away_team
    full_def_stats['Home Score'] = hscore
    full_def_stats['Away Score'] = ascore
    full_def_stats['Texas Result'] = tex_win
    full_def_stats['Link'] = url
    full_def_stats['Season'] = year_frame

    # now add it to the master stats
    master_stats_3 = pd.concat([master_stats_3, full_game_stats], ignore_index = True)
    master_def_3 = pd.concat([master_def_3, full_def_stats], ignore_index = True)

    # finally empty out the full game dataframe
    full_game_data = pd.DataFrame() 

master_stats_3.to_csv("master_stats_3.csv", index = False)
master_def_3.to_csv("master_def_3.csv", index = False)

driver.quit()

Database building...: 100%|██████████| 114/114 [01:07<00:00,  1.68it/s]


get glitched blue box scores kansas and nebraska 2009 (probably wouldn't have bothered with these if I knew it was just two games but its nice not to do manually)

In [18]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
from IPython.display import display, HTML
from tqdm import tqdm
import time
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

# set up master dataframe
stats = {
    'Player': [],
    'Completions': [],
    'Pass Attempts': [],
    'Interceptions': [],
    'Pass Yards': [],
    'Passing TDs': [],
    'Longest Pass': [],
    'Sacks Taken': [],
    'Rush Attempts': [],
    'Rush Yards Gained': [],
    'Rush Yards Lost': [],
    'Net Rush Yards': [],
    'Rushing TDs': [],
    'Longest Rush': [],
    'Yards Per Rush': [],
    'Catches': [],
    'Receiving Yards': [],
    'Receiving TDs': [],
    'Longest Reception': [],
    'GameID': [],
    'Link': []
}
master_stats_4 = pd.DataFrame(stats)

defs = {'##' : [], 
       'Last' : [], 
       'First' : [], 
       'Solo' : [], 
       'Ast' : [], 
       'Tot' : [], 
       'TFL' : [], 
       'tfl_yds' : [], 
       'FF' : [],
       'FR' : [], 
       'fr_yd' : [], 
       'Int' : [], 
       'int_yds' : [], 
       'BrUp' : [], 
       'Blkd' : [], 
       'Sack' : [], 
       'sack_yds' : [],
       'QH' : []
}
master_def_4 = pd.DataFrame(defs)

link_list = ['https://stats.texassports.com/custompages/sports/m-footbl/2009-2010/big12fb.html', # nebraska 2009
             'https://stats.texassports.com/custompages/sports/m-footbl/2009-2010/ut11.html'] # kansas 2009

for i in tqdm(range(len(link_list)), desc = "Database building..."):
    link = link_list[i]
    response = make_request(link)
    temp_box_soup = BeautifulSoup(response, 'html.parser')

    if response == '':
        driver.get(link)
        # driver.get('https://stats.texassports.com/custompages/sports/m-footbl/2009-2010/ut11.html') # for troubleshooting
        temp_box_soup = BeautifulSoup(driver.page_source)

    temp_text = temp_box_soup.text
    temp_text = temp_text.replace("Texas Longhorns", "Texas")
    temp_text = temp_text.replace('TEXAS', 'Texas')

    # Get game date
    date_index = temp_text.find('Date: ')
    date_endex = temp_text.find("Site: ")
    date = temp_text[date_index + 6: date_endex - 3].strip()
    date = date.replace(",", "")
    date = date[:-4] + ',' + date[-4:]
    date = date.replace("Sept", "Sep")
    date = date.replace(" ", "")
    try:
        date = datetime.strptime(date, "%b%d,%Y")
    except ValueError:
        date = datetime.strptime(date, "%b.%d,%Y")      

    tables = temp_box_soup.find_all("table")
    score_table = tables[3]
    away_t_row = 1
    home_t_row = 2

    away_t_col = 0
    home_t_col = 0

    away_s_col = 5
    home_s_col = 5

    home_team = score_table.find_all('tr')[home_t_row].find_all('td')[home_t_col].text.strip().lower()
    away_team = score_table.find_all('tr')[away_t_row].find_all('td')[away_t_col].text.strip().lower()
    ascore = float(score_table.find_all('tr')[away_t_row].find_all('td')[away_s_col].text.strip())
    hscore = float(score_table.find_all('tr')[home_t_row].find_all('td')[home_s_col].text.strip())

    # did texas win?
    if (home_team == 'texas' and hscore > ascore) or (away_team == 'texas' and hscore < ascore):
        tex_win = "Win"
    elif (home_team == 'texas' and hscore < ascore) or (away_team == 'texas' and hscore > ascore):
        tex_win = "Loss"
    else:
        tex_win = "Tie"     

    if home_team == 'texas':
        rush_table = tables[11]
        pass_table = tables[13]
        rec_table = tables[15]
    else:
        rush_table = tables[10]
        pass_table = tables[12]
        rec_table = tables[14]

    # rush stats
    rush_2d = []
    temp = []
    for row in rush_table.find_all('tr')[1:]:
        for val in row.find_all('td'):
            temp.append(val.text.strip())
        if temp[0] == '':
            pass
        else:
            rush_2d.append(temp)
        temp = []

    rush_col = ['Player', 'Rush Attempts', 'Rush Yards Gained', 'Rush Yards Lost', 'Net Rush Yards', 'Rushing TDs', 'Longest Rush', 'Yards Per Rush']

    rush_data = pd.DataFrame(rush_2d)
    rush_data.columns = rush_col

    # pass stats
    pass_2d = []
    temp = []
    for row in pass_table.find_all('tr')[1:]:
        for val in row.find_all('td'):
            temp.append(val.text.strip())
        if temp[0] == '':
            pass
        else:
            pass_2d.append(temp)
        temp = []

    for row in pass_2d:
        new_element = row.pop(1).split('-')
        row[1:1] = new_element

    pass_col = ['Player', 'Completions', 'Pass Attempts', 'Interceptions', 'Pass Yards', 'Passing TDs', 'Longest Pass', 'Sacks Taken']

    pass_data = pd.DataFrame(pass_2d)
    pass_data.columns = pass_col

    # pass stats
    rec_2d = []
    temp = []
    for row in rec_table.find_all('tr')[1:]:
        for val in row.find_all('td'):
            temp.append(val.text.strip())
        if temp[0] == '':
            pass
        else:
            rec_2d.append(temp)
        temp = []

    rec_col = ['Player', 'Catches', 'Receiving Yards', 'Receiving TDs', 'Longest Reception']
            
    frames = [pass_2d, rush_2d, rec_2d]
    for frame in frames:
        for i in range(0,len(frame)):
            for j in range(1,len(frame[i])):
                frame[i][j] = float(frame[i][j])

    rec_data = pd.DataFrame(rec_2d)
    rec_data.columns = rec_col

    full_game_data = pd.merge(pass_data, rush_data, how = "outer", on = "Player")
    full_game_data = pd.merge(full_game_data, rec_data, how = 'outer', on = "Player")
    full_game_data = full_game_data.fillna(0)

    full_game_data = full_game_data.replace('Totals...', 'Total')

    gameid = away_team.replace(" ", "").lower() + '_' + home_team.replace(" ", "").lower() + '_' + str(date.month) + '_' + str(date.day) + '_' + str(date.year)
    gameid_list = [gameid]

    def_game_data = def_scrape_4(tables, home_team)

    full_game_data['GameID'] = gameid
    full_game_data['Date'] = date
    full_game_data['Home Team'] = home_team
    full_game_data['Away Team'] = away_team
    full_game_data['Home Score'] = hscore
    full_game_data['Away Score'] = ascore
    full_game_data['Texas Result'] = tex_win
    full_game_data['Link'] = link
    full_game_data['Season'] = '2009'

    def_game_data['GameID'] = gameid
    def_game_data['Date'] = date
    def_game_data['Home Team'] = home_team
    def_game_data['Away Team'] = away_team
    def_game_data['Home Score'] = hscore
    def_game_data['Away Score'] = ascore
    def_game_data['Texas Result'] = tex_win
    def_game_data['Link'] = link
    def_game_data['Season'] = '2009'

    numeric_cols = full_game_data.select_dtypes(include='number').columns
    full_game_data[numeric_cols] = full_game_data[numeric_cols].astype(float)

    # now add it to the master stats
    master_stats_4 = pd.concat([master_stats_4, full_game_data], ignore_index = True)
    master_def_4 = pd.concat([master_def_4, def_game_data], ignore_index=True)

    # finally empty out the full game dataframe
    full_game_data = pd.DataFrame() 

master_stats_4.to_csv("master_stats_4.csv", index = False)
master_def_4.to_csv("master_def_4.csv", index = False)

Database building...: 100%|██████████| 2/2 [00:03<00:00,  1.70s/it]


2008 (i do not like 2008
        i do not like it in a gate
            i do not like it when it skate
                i do not like 2008)

In [64]:
from selenium import webdriver
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.chrome.service import Service
from bs4 import BeautifulSoup
import pandas as pd
from datetime import datetime
import warnings
from IPython.display import display, HTML
from tqdm import tqdm
import time
import requests
from urllib3.util.retry import Retry
from requests.adapters import HTTPAdapter

# set up master dataframe
stats = {
    'Player': [],
    'Completions': [],
    'Pass Attempts': [],
    'Interceptions': [],
    'Pass Yards': [],
    'Passing TDs': [],
    'Longest Pass': [],
    'Sacks Taken': [],
    'Rush Attempts': [],
    'Rush Yards Gained': [],
    'Rush Yards Lost': [],
    'Net Rush Yards': [],
    'Rushing TDs': [],
    'Longest Rush': [],
    'Yards Per Rush': [],
    'Catches': [],
    'Receiving Yards': [],
    'Receiving TDs': [],
    'Longest Reception': [],
    'GameID': [],
    'Link': []
}
master_stats_5 = pd.DataFrame(stats)

defs = {'##' : [], 
       'Last' : [], 
       'First' : [], 
       'Solo' : [], 
       'Ast' : [], 
       'Tot' : [], 
       'TFL' : [], 
       'tfl_yds' : [], 
       'FF' : [],
       'FR' : [], 
       'fr_yd' : [], 
       'Int' : [], 
       'int_yds' : [], 
       'BrUp' : [], 
       'Blkd' : [], 
       'Sack' : [], 
       'sack_yds' : [],
       'QH' : []
}
master_def_5 = pd.DataFrame(defs)

link_list = ['http://stats.texassports.com/sports/m-footbl/2008-2009/ut1.html',
             'http://stats.texassports.com/sports/m-footbl/2008-2009/ut2.html',
             'http://stats.texassports.com/sports/m-footbl/2008-2009/ut3.html',
             'http://stats.texassports.com/sports/m-footbl/2008-2009/ut4.html',
             'http://stats.texassports.com/sports/m-footbl/2008-2009/ut5.html',
             'http://stats.texassports.com/sports/m-footbl/2008-2009/ut6.html',
             'http://stats.texassports.com/sports/m-footbl/2008-2009/ut7.html',
             'http://stats.texassports.com/sports/m-footbl/2008-2009/ut8.html',
             'http://stats.texassports.com/sports/m-footbl/2008-2009/ut9.html',
             'http://stats.texassports.com/sports/m-footbl/2008-2009/ut10.html',
             'http://stats.texassports.com/sports/m-footbl/2008-2009/ut11.html',
             'http://stats.texassports.com/sports/m-footbl/2008-2009/ut12.html',            
             'http://stats.texassports.com/sports/m-footbl/2008-2009/ut13.html']

for i in tqdm(range(len(link_list)), desc = "Database building..."):
    # Get full page soup
    link = link_list[i]
    response = make_request(link)
    temp_box_soup = BeautifulSoup(response, 'html.parser')

    if response == '':
        driver.get(link)
        # driver.get('https://stats.texassports.com/custompages/sports/m-footbl/archive/stats/06/ut11.htm') # for troubleshooting
        temp_box_soup = BeautifulSoup(driver.page_source)

    temp_text = temp_box_soup.text
    temp_text = temp_text.replace("Texas Longhorns", "Texas")
    temp_text = temp_text.replace('TEXAS', 'Texas')

    # Get game date
    date_index = temp_text.find('Date: ')
    date_endex = temp_text.find("Site: ")
    date = temp_text[date_index + 6: date_endex].strip()
    if date == "0ct 10, 1959":
        date = "Oct 10, 1959"
    date = date.replace(",", "")
    date = date[:-4] + ',' + date[-4:]
    date = date.replace("Sept", "Sep")
    date = date.replace(" ", "")
    try:
        date = datetime.strptime(date, "%b%d,%Y")
    except ValueError:
        date = datetime.strptime(date, "%b.%d,%Y")      

    # Get away team
    away_index = temp_text.find('Photo Gallery')
    away_endex = temp_text.find(' vs ')
    away_team = temp_text[away_index + 13: away_endex].strip().lower()
    if away_team == "xas":
        away_team = "texas"
    elif away_team == "ylor":
        away_team = "baylor"

    # Get home team
    home_index = temp_text.find(' vs ')
    home_endex = temp_text.find(' (')
    home_team = temp_text[home_index + 4: home_endex].strip().lower()

    # get away score
    temp_index = temp_text.find('Score by Quarters     1  2  3  4   Score')
    temp_text_new = temp_text[temp_index:]
    temp_index = temp_text_new.find('\n')
    temp_text_new = temp_text_new[temp_index + 1:]
    temp_index = temp_text_new.find('\n')
    temp_text_new = temp_text_new[temp_index + 1:]
    ascore_index = temp_text_new.find(' - ') + 3
    ascore_endex = temp_text_new.find('\n')
    ascore = temp_text_new[ascore_index:ascore_endex].strip()
    ascore = float(ascore)

    # get home score
    temp_text_new = temp_text_new[ascore_endex + 1:]
    hscore_index = temp_text_new.find(' - ') + 3
    hscore_endex = temp_text_new.find('\n')
    hscore = temp_text_new[hscore_index:hscore_endex].strip()
    hscore = float(hscore)

    # did texas win?
    if (home_team == 'texas' and hscore > ascore) or (away_team == 'texas' and hscore < ascore):
        tex_win = "Win"
    elif (home_team == 'texas' and hscore < ascore) or (away_team == 'texas' and hscore > ascore):
        tex_win = "Loss"
    else:
        tex_win = "Tie"

    # get UT box score text
    temp = temp_box_soup.text
    index = temp.find("Individual Statistics")
    temp = temp[index + 21:]
    index = temp.find("Individual Statistics")
    temp = temp[index + 21:]
    index = temp.find("Individual Statistics")
    temp = temp[index + 21:]
    temp = temp.replace('Texas Longhorns', 'Texas')
    temp = temp.replace('TEXAS', 'Texas')

    split = temp.find('Field goal attempts')
    if home_team == 'texas': 
        temp = temp[split:]
    else: 
        temp = temp[:split]

    # Truncate box score for rushing stats
    rush_start = temp.find('Rushing              No Gain Loss  Net TD Lg  Avg')
    rush_end = temp.find('Passing              ') - 2
    rush_temp = temp[rush_start:rush_end]

    # Get rushing stats
    rush_stats = []    
    line_break = rush_temp.find('\n')
    header = rush_temp[0:line_break].split()
    rush_stats.append(header)
    rush_temp = rush_temp[line_break:]
    line_break = rush_temp.find('\n')
    rush_temp = rush_temp[line_break + 1:]

    # Now rush temp has no header
    line_break = rush_temp.find('\n')
    rush_temp = rush_temp[line_break + 1:]
    while True:
        line_break = rush_temp.find('\n')
        line = rush_temp[0:line_break + 1]
        game_stat = line.split()
        if rush_temp.find('\n') == -1:
            line = rush_temp
            game_stat = line.split()          
            rush_stats.append(game_stat)
            rush_temp = rush_temp[line_break + 1:]
            break
        else:
            # combines first n name columns
            while len(game_stat) > 8: 
                name = game_stat[0] + ' ' + game_stat[1]
                stats = game_stat[2:]
                game_stat = [name] + stats
            rush_stats.append(game_stat)
            rush_temp = rush_temp[line_break + 1:]
    
    # Truncate box score for passing stats
    pass_start = temp.find('Passing              ')
    pass_end = temp.find('Receiving             No.  Yds   TD Long') - 3
    pass_temp = temp[pass_start:pass_end]

    # Get passing stats
    pass_stats = []    
    line_break = pass_temp.find('\n')
    header = pass_temp[0:line_break].split()
    pass_stats.append(header)
    pass_temp = pass_temp[line_break:]
    line_break = pass_temp.find('\n')
    pass_temp = pass_temp[line_break + 1:]

    # Now pass temp has no header
    line_break = pass_temp.find('\n')
    pass_temp = pass_temp[line_break + 1:]
    while True:
        line_break = pass_temp.find('\n')
        line = pass_temp[0:line_break + 1]
        game_stat = line.split()
        if pass_temp.find('\n') == -1:
            line = pass_temp
            game_stat = line.split()
            pass_stats.append(game_stat)
            pass_temp = pass_temp[line_break + 1:]
            break
        else:
            # combines first n name columns
            while len(game_stat) > 6: 
                name = game_stat[0] + ' ' + game_stat[1]
                stats = game_stat[2:]
                game_stat = [name] + stats
            pass_stats.append(game_stat)
            pass_temp = pass_temp[line_break + 1:]
    
    # Truncate box score for rec stats
    rec_start = temp.find('Receiving             No.  Yds   TD Long')
    rec_end = temp.find('Punting               No.  Yds   Avg Long In20   TB')
    rec_temp = temp[rec_start:rec_end]

    # Get rec stats
    rec_stats = []    
    line_break = rec_temp.find('\n')
    header = rec_temp[0:line_break].split()
    rec_stats.append(header)
    rec_temp = rec_temp[line_break:]
    line_break = rec_temp.find('\n')
    rec_temp = rec_temp[line_break + 1:]

    # Now rec temp has no header
    line_break = rec_temp.find('\n')
    rec_temp = rec_temp[line_break + 1:]
    while True:
        line_break = rec_temp.find('\n')
        line = rec_temp[0:line_break + 1]
        game_stat = line.split()
        if rec_temp.find('\n') == -1:
            line = rec_temp
            game_stat = line.split()
            rec_stats.append(game_stat)
            rec_temp = rec_temp[line_break + 1:]
            break
        else:
            # combines first n name columns
            while len(game_stat) > 5: 
                name = game_stat[0] + ' ' + game_stat[1]
                stats = game_stat[2:]
                game_stat = [name] + stats
            rec_stats.append(game_stat)
            rec_temp = rec_temp[line_break + 1:]

    ##############################################################
    # Now that we have the stats in 2d lists, we need to make sure they aren't just strings
    # First, we must address the formatting of the passing cmp-att-int format
    for row in pass_stats:
        new_element = row.pop(1).split('-')
        row[1:1] = new_element
    
    # Next, we must make sure the elements are floats and not strings
    frames = [pass_stats, rush_stats, rec_stats]
    for frame in frames:
        for i in range(1,len(frame)):
            for j in range(1,len(frame[i])):
                frame[i][j] = float(frame[i][j])
                
    # Now, we make the arrays into dataframes using panda
    # admittedly i shouldve done this earlier but oh well
    rush_data = pd.DataFrame(rush_stats[1:])
    rush_data.columns = ['Player', 'Rush Attempts', 'Rush Yards Gained', 'Rush Yards Lost', 'Net Rush Yards', 'Rushing TDs', 'Longest Rush', 'Yards Per Rush']

    pass_data = pd.DataFrame(pass_stats[1:])
    pass_data.columns = ['Player', 'Pass Attempts', 'Completions', 'Interceptions', 'Pass Yards', 'Passing TDs', 'Longest Pass', 'Sacks Taken']

    rec_data = pd.DataFrame(rec_stats[1:])
    rec_data.columns = ['Player', 'Catches', 'Receiving Yards', 'Receiving TDs', 'Longest Reception']
    
    # Finally, time to merge the data into one full dataframe for the full game
    full_game_data = pd.merge(
        pass_data, rush_data, how = "outer", on = "Player"
    )
    full_game_data = pd.merge(
        full_game_data, rec_data, how = 'outer', on = "Player"
    )
    full_game_data = full_game_data.fillna(0)

    # small thing but i want to take the ellipsis out of the totals category
    full_game_data = full_game_data.replace('Totals...', 'Total')

    # now make the gameid
    gameid = away_team.replace(" ", "").lower() + '_' + home_team.replace(" ", "").lower() + '_' + str(date.month) + '_' + str(date.day) + '_' + str(date.year)
    gameid_list = [gameid]

    '''
    game_df = {'Home Team' : [home_team],
            'Away Team' : [away_team],
            'Home Score' : [hscore],
            'Away Score' : [ascore],
            'Texas Result' : [tex_win],
            'Box Score' : [full_game_data]
            }
    
    game_df = pd.DataFrame(game_df, index = gameid_list)

    # finally append it to the master games
    master_games = pd.concat([master_games, game_df], ignore_index = True)

    # empty out game_df
    game_df = pd.DataFrame()
    '''
    temp = temp_box_soup.text
    def_game_data = def_scrape_5(temp, home_team)

    ##############################################################
    # the last thing I want to do is to create one large dataframe with every single game performance ever
    # this will contain duplicate players for their different performances in different games
    # much less concise, much more usefull (probably)
    # first add gameid column
    full_game_data['GameID'] = gameid
    full_game_data['Date'] = date
    full_game_data['Home Team'] = home_team
    full_game_data['Away Team'] = away_team
    full_game_data['Home Score'] = hscore
    full_game_data['Away Score'] = ascore
    full_game_data['Texas Result'] = tex_win
    full_game_data['Link'] = link
    full_game_data['Season'] = '2008'

    def_game_data['GameID'] = gameid
    def_game_data['Date'] = date
    def_game_data['Home Team'] = home_team
    def_game_data['Away Team'] = away_team
    def_game_data['Home Score'] = hscore
    def_game_data['Away Score'] = ascore
    def_game_data['Texas Result'] = tex_win
    def_game_data['Link'] = link
    def_game_data['Season'] = '2008'

    # now add it to the master stats
    master_stats_5 = pd.concat([master_stats_5, full_game_data], ignore_index = True)
    master_def_5 = pd.concat([master_def_5, def_game_data], ignore_index = True)

    # finally empty out the full game dataframe
    full_game_data = pd.DataFrame() 
        
# print(master_stats_1)
master_stats_5 = master_stats_5[master_stats_5["Player"] != 0]
master_stats_5.to_csv('master_stats_5.csv', index = False)
master_def_5.to_csv('master_def_5.csv', index = False)
# master_games.to_csv('master_games.csv', index = False)

Database building...: 100%|██████████| 13/13 [00:11<00:00,  1.18it/s]


2023 and beyond

In [ ]:
# maybe i can adapt the old code one moment

things i need to fix:
- north texas glitch [done]
- la tech 2019 (maybe this will run now) [done]
- 3 games in 1998 [done]
- arkansas 2014 [done]
- add link to blue tables and new format [done]
- 2008, kansas 2009, nebraska 2009 [done]

merge the master stats

In [71]:
master_stats_1 = pd.read_csv('master_stats_1.csv')
master_stats_2 = pd.read_csv('master_stats_2.csv')
master_stats_3 = pd.read_csv('master_stats_3.csv')
master_stats_4 = pd.read_csv('master_stats_4.csv')
master_stats_5 = pd.read_csv('master_stats_5.csv')

master_stats = pd.concat([master_stats_1, 
                          master_stats_2, 
                          master_stats_3, 
                          master_stats_4, 
                          master_stats_5], 
                          ignore_index=True)
master_stats.to_csv('master_stats.csv', index = False)

master_def_1 = pd.read_csv('master_def_1.csv')
master_def_2 = pd.read_csv('master_def_2.csv')
master_def_3 = pd.read_csv('master_def_3.csv')
master_def_4 = pd.read_csv('master_def_4.csv')
master_def_5 = pd.read_csv('master_def_5.csv')

master_def = pd.concat([master_def_1, 
                          master_def_2, 
                          master_def_3, 
                          master_def_4, 
                          master_def_5], 
                          ignore_index=True)
master_def.to_csv('master_def.csv', index = False)

clean up
- give first and last name columns
- strip the names
- put names in front

In [55]:
import pandas as pd

def split_names(row):
    full_name = row['Last Name']
    if pd.notna(full_name) and pd.isna(row['First Name']):
        names = full_name.split()
        if len(names) == 2:
            row['First Name'] = names[0]
            row['Last Name'] = names[1]
    return row

master_stats_test = pd.read_csv('master_stats.csv')

# who the fuck did this to johnny walker 4 times i do not understand
master_stats_test['Player'].replace('Walker. Johnny', 'Walker, Johnny', inplace = True)

# split on commas
master_stats_test[['Last Name', 'First Name']] = master_stats_test['Player'].str.split(pat=',', n=1, expand=True)
master_stats_test = master_stats_test.drop('Player', axis=1)

# strip the names
master_stats_test['Last Name'] = master_stats_test['Last Name'].str.strip()
master_stats_test['First Name'] = master_stats_test['First Name'].str.strip()

# fix players with "First Last" Format
master_stats_test = master_stats_test.apply(split_names, axis=1) 

# order the cols
front_columns = ['First Name', 'Last Name']
master_stats_test = master_stats_test[front_columns + [col for col in master_stats_test.columns if col not in front_columns]]

# change totals to total
master_stats_test['Last Name'].replace("Total", "Game", inplace = True)
master_stats_test['Last Name'].replace("Totals", "Game", inplace = True)

master_stats_test.drop_duplicates(inplace = True)

master_stats_test.to_csv('master_stats_test.csv', index = False)

make player IDs - offense

In [8]:
import pandas as pd
master_stats_test = pd.read_csv('master_stats_test.csv')

# house keeping
master_stats_test.loc[master_stats_test['First Name'] == 'Jor', 'First Name'] = 'Jordan'
master_stats_test.at[134, 'First Name'] = 'Missing Name'
master_stats_test.at[808, 'First Name'] = 'My Main Man'
master_stats_test.at[4781, 'First Name'] = 'Missing Name'
master_stats_test['First Name'] = master_stats_test['First Name'].replace('Lil\'J', 'Lil\'Jordan')

master_stats_test['Date'] = pd.to_datetime(master_stats_test['Date'])
master_stats_test['Year'] = master_stats_test['Date'].dt.year.astype(int)
master_stats_test = master_stats_test.sort_values(by='Date').reset_index(drop=True)
master_stats_test['PlayerID'] = ''
master_stats_test['NameConcat'] = ''
master_stats_test['First Year'] = ''
master_stats_test['Last Year'] = ''
master_stats_test['Opponent'] = ''
master_stats_test['Score'] = master_stats_test['Texas Result'] + ' ' + master_stats_test['Home Score'].astype(int).astype(str) + '-' + master_stats_test['Away Score'].astype(int).astype(str)

for i in range(len(master_stats_test)):
    if master_stats_test['Home Team'][i] == 'texas':
        master_stats_test.loc[i, 'Opponent'] = master_stats_test['Away Team'][i]
    else:
        master_stats_test.loc[i, 'Opponent'] = master_stats_test['Home Team'][i]

# this is a bad bad inefficient piece of code but it adds the concatonated name
for i in range(len(master_stats_test)):
    try:
        float(master_stats_test['First Name'][i])

        if master_stats_test['Last Name'][i] == 'Game':
            master_stats_test.loc[i, 'NameConcat'] = 'Game'
        else:
            master_stats_test.loc[i, 'NameConcat'] = 'Team'
    except:
        master_stats_test.loc[i, 'NameConcat'] = master_stats_test.loc[i, 'First Name'] + master_stats_test.loc[i, 'Last Name']

# check if each row is a player or a game, then give the players ids if they don't have them
for i in range(len(master_stats_test)):
    try:
        float(master_stats_test['First Name'][i])
        pass
    except:
        if master_stats_test['PlayerID'][i] == '':
            name = master_stats_test['First Name'][i] + master_stats_test['Last Name'][i]
            start = master_stats_test['Year'][i] - 1
            end = master_stats_test['Year'][i] + 6

            conditions = (
                (master_stats_test['Year'] < end) &
                (master_stats_test['Year'] >= start) &
                (master_stats_test['NameConcat'] == name)
            )

            master_stats_test.loc[conditions, 'PlayerID'] = i + 1
            master_stats_test.loc[conditions, 'First Year'] = master_stats_test.loc[conditions, 'Season'].min()
            master_stats_test.loc[conditions, 'Last Year'] = master_stats_test.loc[conditions, 'Season'].max()

master_stats_test['PlayerID'] = master_stats_test['PlayerID'].astype(str).str.zfill(6)
master_stats_test['Fantasy'] = ((-2 * master_stats_test['Interceptions']) + 
                                (.04 * master_stats_test['Pass Yards']) + 
                                (6 * master_stats_test['Passing TDs']) + 
                                (.1 * master_stats_test['Net Rush Yards']) + 
                                (6 * master_stats_test['Rushing TDs']) + 
                                (.5 * master_stats_test['Catches']) + 
                                (.1 * master_stats_test['Receiving Yards']) + 
                                (6 * master_stats_test['Receiving TDs'])
)

master_stats_test.to_csv('master_stats_final.csv', index = False)

In [8]:
x = pd.read_csv('master_stats_final.csv')

x = x.groupby('Date').sum(numeric_only=True)
y = x[x['Completions'] != x['Catches']]
display(y)

z = x[x['Pass Yards'] != x['Receiving Yards']]
display(z)

,Completions,Pass Attempts,Interceptions,Pass Yards,Passing TDs,Longest Pass,Sacks Taken,Rush Attempts,Rush Yards Gained,Rush Yards Lost,...,Receiving TDs,Longest Reception,Home Score,Away Score,Season,Year,PlayerID,First Year,Last Year,Fantasy
Date,,,,,,,,,,,,,,,,,,,,,
1989-11-18,38.0,64.0,2.0,482.0,2.0,105.0,6.0,84.0,502.0,72.0,...,2.0,175.0,403.0,221.0,25857,25857,59786,23853.0,23885.0,183.68


,Completions,Pass Attempts,Interceptions,Pass Yards,Passing TDs,Longest Pass,Sacks Taken,Rush Attempts,Rush Yards Gained,Rush Yards Lost,...,Receiving TDs,Longest Reception,Home Score,Away Score,Season,Year,PlayerID,First Year,Last Year,Fantasy
Date,,,,,,,,,,,,,,,,,,,,,
1948-10-23,12.0,32.0,2.0,246.0,2.0,0.0,0.0,86.0,374.0,58.0,...,2.0,181.0,84.0,240.0,23376,23376,575,21420.0,21428.0,116.24
1960-12-17,14.0,34.0,2.0,236.0,0.0,0.0,0.0,90.0,270.0,22.0,...,0.0,105.0,30.0,30.0,19600,19600,12363,17632.0,17650.0,58.84
1980-09-27,22.0,42.0,2.0,348.0,2.0,75.0,0.0,104.0,610.0,26.0,...,2.0,158.0,525.0,0.0,29700,29700,53047,27704.0,27744.0,185.92
1980-10-11,12.0,34.0,4.0,198.0,0.0,50.0,0.0,132.0,522.0,24.0,...,0.0,107.0,117.0,180.0,17820,17820,29958,15828.0,15847.0,99.72
1982-12-25,12.0,46.0,2.0,100.0,0.0,28.0,0.0,88.0,274.0,114.0,...,0.0,56.0,100.0,260.0,19820,19820,35384,17819.0,17846.0,31.80
1989-11-18,38.0,64.0,2.0,482.0,2.0,105.0,6.0,84.0,502.0,72.0,...,2.0,175.0,403.0,221.0,25857,25857,59786,23853.0,23885.0,183.68
1994-09-10,46.0,76.0,4.0,494.0,2.0,88.0,0.0,76.0,370.0,62.0,...,2.0,192.0,360.0,192.0,23928,23928,61671,21918.0,21940.0,153.36
1994-11-24,38.0,54.0,0.0,608.0,10.0,105.0,0.0,88.0,576.0,20.0,...,10.0,203.0,455.0,819.0,25922,25922,67389,23911.0,23938.0,329.72
1996-12-07,38.0,58.0,4.0,706.0,2.0,132.0,0.0,56.0,330.0,30.0,...,2.0,290.0,370.0,270.0,19960,19960,53040,17953.0,17971.0,199.24
